In [2]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "#1f1e1e"}

import io
import os
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import Markdown, display, Image
from PIL import Image as PILImage

# LOCAL GIT REPOSITORY LOCATION FOR https://github.com/CSSEGISandData/COVID-19
local_covid_19_data = '/Users/davidmorton/Documents/COVID-19/'

!git -C $local_covid_19_data pull

states_to_drop = ['Puerto Rico', 'American Samoa', 'Guam', 'District of Columbia', 'Northern Mariana Islands', 'Virgin Islands']

state_names = pd.read_csv('Data/states.csv').set_index('State').to_dict()['Abbreviation']

Already up to date.


In [3]:
def get_single_date_data(date):
    datestr = date.strftime('%m-%d-%Y')
    filename = local_covid_19_data + 'csse_covid_19_data/csse_covid_19_daily_reports_us/' + datestr + '.csv'
    result = pd.read_csv(filename)
    result['Date'] = date
    return result

def get_daily_report_data():
    start_date_for_daily_data = datetime(2020, 4, 12)
    current_date = start_date_for_daily_data

    yesterday = (datetime.now() + timedelta(days = -1))
    #yesterday = datetime.now()
    lastdate = yesterday.strftime('%m-%d-%Y')
    
    series = pd.DataFrame()

    while True:
        series = series.append(get_single_date_data(current_date))
        current_date = current_date + timedelta(days = 1)
        if current_date > yesterday:
            break
    
    
    return series

def get_heatmap_data():
    states_to_drop = ['Puerto Rico', 'American Samoa', 'Guam', 'Northern Mariana Islands', 'Virgin Islands', 'Diamond Princess', 'Grand Princess', 'Recovered']

    # Get the daily report, and immediately sort it by province and date, and remove the non-state territories. 
    data = get_daily_report_data()[['Province_State', 'Date', 'Confirmed', 'People_Tested']]
    data = data.sort_values(by=['Province_State', 'Date']).reset_index()
    data = data[~data['Province_State'].isin(states_to_drop)]
    data = data.set_index(['Province_State', 'Date'])
    
    # Create a confirmed percentage column, and then roll it in a 7-day rolling average
    data['ConfirmedPercent'] = data['Confirmed'].div(data['People_Tested'])
    data['ConfirmedPercentRolling'] = data.sort_values(by=['Province_State', 'Date']).groupby('Province_State')['ConfirmedPercent'].rolling(window=7).mean().reset_index(level=1, drop=1)

    # Create a pivot table for use in the heatmap.
    data = data.reset_index()[['Province_State', 'Date', 'ConfirmedPercentRolling']].pivot(values='ConfirmedPercentRolling', index='Date', columns='Province_State')
    data = data.reindex(data.mean().sort_values(ascending=False).index, axis=1).dropna()

    return data

def draw_figure():
    xxp = get_heatmap_data()
    
    ylabels = [' {0} ({1:.2f}%) '.format(a[0], a[1]*100) for a in zip(xxp.columns, xxp.mean().sort_values(ascending=False))]
    xlabels = [list(reversed(pd.Series(xxp.index)))[i].strftime('%B %-d') if i % 7==0 else '' for i in np.arange(len(xxp.index)-1,-1, -1)]

    plt.style.use('dark_background')
    plt.subplots(figsize=(18,23))

    ax = sns.heatmap(xxp.transpose() * 100, 
                     vmin=0, 
                     vmax=20, 
                     cmap='icefire')

    ax.set_xticklabels(labels=xlabels, rotation=45, ha='right', size=14)
    ax.set_xlabel('')
    ax.set_yticklabels(labels=ylabels, size=14)
    ax.set_ylabel('')
    ax.tick_params(axis='both', which='both', length=0)

    ax.collections[0].colorbar.ax.tick_params(labelsize=14)

    ax.set_title('7-Day Rolling Average of Positive COVID-19\nTest Results Sorted by State Average', size=36, pad=20, fontname='Helvetica')
    
    plt.annotate('Source: Johns Hopkins COVID-19 Data (https://github.com/CSSEGISandData/COVID-19)', (0,0), (100, -90), xycoords='axes fraction', fontname='Helvetica', textcoords='offset points', size=14, va='top')
    plt.savefig("covid19-confirmed-heatmap.png", bbox_inches="tight", pad_inches=0.5)
    
def make_usa_choropleth(series, locations, title, colorbartitle, colorspectrum, zmax):
    

    fig = go.Figure(
        data=go.Choropleth(
            locations=locations,
            z = series,
            locationmode = 'USA-states',
            colorscale = colorspectrum,
            colorbar_title = colorbartitle, 
            zmax=zmax,
            zmin=0
        )
    )

    fig.update_layout(
        title_text = title,
        geo_scope='usa', # limite map scope to USA
    )

    return fig;

def make_animation(totalduration, pausetime, timeinterpolation, theme):
    bgcolor="black"
    data = get_heatmap_data() * 100
    data = data.rename(state_names, axis=1)
    
    vals = data.values.flatten().flatten()
    vals.sort()
    bins = pd.qcut(vals, q=8, retbins=True)[1]
    colorstops = (bins - bins.min()) / (bins.max() - bins.min())
    colorscale = list(zip(colorstops, getattr(px.colors.sequential, theme)))
    zmax = vals.max()

    data = data.resample(rule=timeinterpolation).asfreq().interpolate()
    
    adjduration = totalduration-pausetime;
    perslide = round((adjduration*1000)/len(data));
    repeatlast = round((pausetime*1000)/perslide)
    print('Frames will be shown for {0} milliseconds'.format(perslide))
    
    images = []
    for i in range(len(data)):
        row = data.iloc[i]
        fig = make_usa_choropleth(row, row.index, 'Positive Percentage ' + row.name.strftime('%B %d') + ' ({:.2f}% Average)'.format(row.mean()), '%', colorscale, zmax)
        fig.update_layout(
            plot_bgcolor=bgcolor,
            paper_bgcolor=bgcolor,
            font=dict(color="white"),
            geo = dict(
                lakecolor=bgcolor,
                showlakes=True,
                bgcolor=bgcolor
            )
        )
        img_bytes = fig.to_image(format='png')
        image = PILImage.open(io.BytesIO(img_bytes))

        images.append(image)
        if (i == (len(data) - 1)):
            for j in range(1,repeatlast):
                images.append(image)
        
    url = 'Output\covid19-case-lapse.gif'
    images[0].save(url,
                   save_all=True, append_images=images[1:], dpi=300, optimize=True, duration=perslide, loop=0)
    return url + "?c=" + str(random.randint(0,2e9))


In [4]:
url = make_animation(30, 5, '6H', "Inferno")
Image(url=url)

Frames will be shown for 52 milliseconds
